In [16]:
# Header

header = ("Description", "N=35256", "N=38282")
rows = [
    # ... as above ...
    ("Age, y, median [IQR]", "46 [31,64]", "45 [30,63]"),
    ("SBP, mmHg, median [IQR]", "124 [114,135]", "118 [108,134]"),
    ("DBP, mmHg, median [IQR]", "73 [65,80]", "70 [62,76]"),
    ("Serum Creatinine, mg/dL, median [IAR]", "1 [1,1]", "1 [1,1]"),
    ("eGFR, ml/min/1.73m2, median [IAR]", "96 [79,112]", "118 [99,135]"),
    ("BMI, kg/m2, median [IQR]", "27 [24,31]", "28 [23,33]"),
    ("HBA1c, %, median [IQR]", "6 [5,6]", "5 [5,6]"),
    ("uACR, mg/g, median [IQR]", "6 [4,12]", "8 [5,15]"),
    ("Log uACR, log(mg/g), median [IQR]", "2 [1,2]", "2 [2,3]"),
    ("Glucose, mg/L, median [IQR]", "9 [9,10]", "9 [8,10]"),
    ("Income, $, median [IQR]", "23771 [8479,45024]", "22076 [8001,43148]"),
    ("Diabetes, %", "12", "12"),
    ("Hypertension, %", "31", "32"),
    ("Smoke, %", "57", "36"),
    # Adding additional characteristics as requested
    ("Race, White", "43", "42"),
    ("Race, Black", "23", "24"),
    ("Race, Hispanic", "26", "27"),
    ("Race, Other", "8", "8"),
    ("Race2, White", "37", "35"),
    ("Race2, Black", "23", "23"),
    ("Race2, Hispanic", "23", "25"),
    ("Race2, Asian", "13", "13"),
    ("Race2, Other", "4", "3"),
    ("Educ, K-8", "16", "14"),
    ("Educ, High school, Diploma/equivalent", "42", "42"),
    ("Educ, Some college/associate", "22", "25"),
    ("Educ, College graduate/above", "20", "18"),
    # Adding Years
    ("Year, 1988-1998", "23", "24"),
    ("Year, 1999-2000", "7", "7"),
    # Add more years as needed...
]


# Customize the width of each column (in percentage or pixels, e.g., "20%", "100px")
col1_width = "40%"
col2_width = "30%"
col3_width = "30%"

html_output = '<table style="width: 100%;">'
html_output += '<tr><th style="text-align: left; width: {}; padding: 5px;">{}</th><th style="text-align: center; width: {}; padding: 5px;">{}</th><th style="text-align: center; width: {}; padding: 5px;">{}</th></tr>'.format(col1_width, header[0], col2_width, header[1], col3_width, header[2])

current_category = None
for row in rows:
    description, value1, value2 = row
    style_col1 = 'style="text-align: left; width: {}; padding: 5px;"'.format(col1_width)
    style_col2 = 'style="text-align: center; width: {}; padding: 5px;"'.format(col2_width)
    style_col3 = 'style="text-align: center; width: {}; padding: 5px;"'.format(col3_width)
    
    if description.startswith("Race") or description.startswith("Educ") or description.startswith("Year"):
        if description.split(",")[0] != current_category:
            current_category = description.split(",")[0]
            html_output += '<tr><td {}><b>{}</b></td><td></td><td></td></tr>'.format(style_col1, current_category)
        description = "&nbsp;&nbsp;&nbsp;" + description.split(",")[1]
        
    html_output += '<tr><td {}>{}</td><td {}>{}</td><td {}>{}</td></tr>'.format(style_col1, description, style_col2, value1, style_col3, value2)

html_output += '</table>'

from IPython.core.display import HTML
display(HTML(html_output))

Description,N=35256,N=38282
"Age, y, median [IQR]","46 [31,64]","45 [30,63]"
"SBP, mmHg, median [IQR]","124 [114,135]","118 [108,134]"
"DBP, mmHg, median [IQR]","73 [65,80]","70 [62,76]"
"Serum Creatinine, mg/dL, median [IAR]","1 [1,1]","1 [1,1]"
"eGFR, ml/min/1.73m2, median [IAR]","96 [79,112]","118 [99,135]"
"BMI, kg/m2, median [IQR]","27 [24,31]","28 [23,33]"
"HBA1c, %, median [IQR]","6 [5,6]","5 [5,6]"
"uACR, mg/g, median [IQR]","6 [4,12]","8 [5,15]"
"Log uACR, log(mg/g), median [IQR]","2 [1,2]","2 [2,3]"
"Glucose, mg/L, median [IQR]","9 [9,10]","9 [8,10]"


NHANES 

1. 01_nhdemo_final.do

```stata
cls

*! demographics
*! essence: #41  

quietly {

     capture log close
     global date: di %td date(c(current_date),"DMY")
     quietly log using "03_nhdemo$date.log", replace
     version 12

     global nh3 https://wwwn.cdc.gov/nchs/data/nhanes3/1a/
     global start = clock(c(current_time),"hms")/10^3

	
	 //1988-1994
	     noi di "loading... 1988-1994 from $nh3"
     infix seqn 1-5 dmpstat 11 dmarethn 12 dmaethnr 14 hssex 15 hsageir	18-19 ///
	     hfa8r 1256-1257 hff19r 1409-1410 sdpphase 42 using ${nh3}adult.DAT, clear
	 tempfile surv1
	 gen surv=1
     save `surv1' 

     //1999-2017
     global nhanes "https://wwwn.cdc.gov/Nchs/Nhanes"  
     tokenize "`c(ALPHA)'" //i.e., "`2'" -> B, "`3'" -> C, etc.
	 local Letter=1 //tokens: B-J
     local N=2 //surveys: 1999-2017   
	 
	 foreach y of numlist 1999 2001 2003 2005 2007 2009 2011 2013 2015 2017 {  
	 	
         local yplus1=`y' + 1  
		 if `y'==1999 {
		 	 local letter: di ""
		 }
		 if `y' >2000 {
		     local letter: di "_``Letter''"  
		 }
		 
         import sasxport5 "$nhanes/`y'-`yplus1'/DEMO`letter'.XPT", clear   
	     noi di "loading... `y'-`yplus1' from $nhanes"
	     tempfile surv`N'
		 gen surv=`N'
         save `surv`N'' 
	     local N=`N' + 1
	     local Letter=`Letter' + 1

     }
	 
	 clear
	 noi di "..."
	 noi di "LOADED!"
     forvalues i=1/11 {
	 	 append using `surv`i''
     }

	 compress
	 local sample=100
	 sample `sample'
     lab dat "`sample'% of NHANES 1988-2018, Demographics"
     save "02_nhdemo$date", replace 
	 noi di ""
	 noi di ""
	 noi di "02_nhdemo$date.dta saved to " word("`c(pwd)'",1)
     
	 //big data!
	 noi di ""
	 noi di "rows x columns:"
	 noi count
	 ds 
	 noi di wordcount(r(varlist))
	 
     global finish  = clock(c(current_time),"hms")/10^3
     global duration = round(($finish - $start)/60,1)

     noi di "runtime = $duration minute(s)"
     log close

}




```

---

2. 04_nhexam_final.do

```stata
cls

*! assemble datasets
*! exam, lab, q&a 
*! do so for all nh surveys

quietly {
	
     capture log close
     global date: di %td date(c(current_date),"DMY")
     log using "06_nhexam$date.log", replace
     version 12 

     global nh3 https://wwwn.cdc.gov/nchs/data/nhanes3/1a/
     global nhanes "https://wwwn.cdc.gov/Nchs/Nhanes" 
     global start = clock(c(current_time),"hms")/10^3
	
	 
     //1988-1994
	 noi di "processing... 1988-1994/EXAM.DAT" 
     infix seqn 1-5 pepmnk1r 1423-1425 pepmnk5r 1428-1430 bmpbmi 1524-1527 /// 
	     using ${nh3}exam.dat, clear
     tempfile exam
     save `exam'

	 noi di "processing... 1988-1994/LAB.DAT" 
	 infix seqn 1-5 ghp 1861-1864 ubp 1965-1970 urp 1956-1960 sgpsi 1761-1765 /// 
		 cep 1784-1787 using ${nh3}lab.dat, clear
     tempfile lab
     save `lab'
	 
	 noi di "processing... 1988-1994/ADULT.DAT" 
     infix seqn 1-5 hae4d2 1605 had1 1561 had6 1568 had10 1578 hak1 1855 /// 
	     hae2 1598 hae5a 1610 hat1met 2393-2395 har1 2281 ///
		 using ${nh3}adult.DAT, clear
		 
     tempfile qa 
     save `qa'  
	 
     use `exam', clear 
     merge 1:1 seqn using `lab', nogen
     merge 1:1 seqn using `qa', nogen 
	 	 
     gen surv=1
	 tempfile exam_0
     save `exam_0' 
	 
	 //1999-2018
	 tokenize "`c(ALPHA)'" //i.e., "`2'" -> B, "`3'" -> C, etc.
     local surv=2
	 
	 local y=1999
	 
	 forvalues i=1/10  { //nhanes survey
	 	
		 local yp1=`y'+1
		 
  		 forvalues j=1/11 { //specific datafile
	     
		 local DATA1 BPX BMX LAB10 LAB16  LAB18  ALQ DIQ KIQ   BPQ PAQ SMQ  	
         local DATA2 BPX BMX L10   L16    L40    ALQ DIQ KIQ_U BPQ PAQ SMQ 
         local DATA3 BPX BMX GHB   ALB_CR BIOPRO ALQ DIQ KIQ_U BPQ PAQ SMQ 
		 
		 if `i'==1 {
		 	 local letter: di "" 
			 local DATA `DATA1'
		 }
		 if inlist(`i', 2, 3) {
		 	 local letter: di "_``i''"  
			 local DATA `DATA2'
		 }
		 if inrange(`i', 4,11) {
		 	 local letter: di "_``i''"
			 local DATA `DATA3'
		 }
		 
		 local DATA: di word("`DATA'",`j') 	//jth datafile 
	     import sasxport5 "$nhanes/`y'-`yp1'/`DATA'`letter'.XPT", clear  
		 noi di "processing... `y'-`yp1'/`DATA'`letter'.XPT"
         tempfile dat`j'
         save `dat`j'' 
	
	     }
   
         use `dat1', clear
         forvalues k=2/11 { //merge k datafiles from 1 year
             merge 1:1 seqn using `dat`k'', gen(merge`k')
         }

	     gen surv=`surv'
	     local surv=`surv'+1
	 
	     tempfile exam_`i'
	     save `exam_`i'' //one datafile from ith survey 
		 local y=`y'+2 
		 
     }
	 
	 clear
	 use `exam_0' //1988-1994: nhanes 3
     forvalues i=1/10 {
	 append using `exam_`i'' //1999-2018: continuous nhanes

     }
	 
     lab dat "NHANES 1988-2018, Q&A/Exam/Labs"
	 lookfor smq
     save "05_nhexam$date", replace
	 noi di ""
	 noi di ""
	 noi di "05_nhexam$date.dta saved to " word("`c(pwd)'",1)
	 
	 //big data!
	 noi di ""
	 noi di "rows x columns:"
	 noi count
	 ds 
	 noi di wordcount(r(varlist))
	 
global finish  = clock(c(current_time),"hms")/10^3
global duration = round(($finish - $start)/60,1)

noi di "runtime = $duration minute(s)"

log close
	 
}



*! quietly, loops, indent
*! collapsing, debugging
*! this is a pro-tip :)






``` 

--- 

3. 07_nhmerge_final.do

```stata
cls
macro drop _all

*! merge nhdemo & nhexam 
*! do not rename variables
*! and do not apply selection criteria
*! rename 07_nhtable1.do -> 07_nhmerge.do
*! move blocks of code that rename & select 
*! to new .do file -> 17_nhtable1.do 



quietly {
	
	 capture log close
     global date: di %td date(c(current_date),"DMY")
     log using "09_nhmerge$date.log", replace
     version 12
     global start = clock(c(current_time),"hms")/10^3 
	
     //1.data
	 
     use 05_nhexam$date, clear  
	 replace seqn=-1*seqn if surv==1  //restore unique seqn across surveys 
	 tempfile 05_nhexam  
	 save `05_nhexam'
	
     use 02_nhdemo$date, clear
	 replace seqn=-1*seqn if surv==1
	 noi merge 1:1 seqn using `05_nhexam' //enforce a strictly 1:1 merge 
	 
	 keep if _merge==3
	 
	 	 //survey
	 rename surv survey //exception to the "no-rename" policy
	 lab var survey "Survey"
	 label define Survey ///
	     1 "1988-1994" ///
		 2 "1999-2000" ///
		 3 "2001-2002" ///
		 4 "2003-2004" ///
		 5 "2005-2006" ///
		 6 "2007-2008" ///
		 7 "2009-2010" ///
		 8 "2011-2012" ///
		 9 "2013-2014" ///
		 10 "2015-2016" ///
		 11 "2017-2018" 
	 label values survey Survey

}

quietly {
	
	 //3.keep variables 
	     keep seqn dmpstat ridstatr surv /// id, adult, survey 
		      hsageir ridageyr hssex riagendr /// age, sex 
		      dmarethn ridreth1 ridreth3 /// race-ethnicity 
		      hfa8r dmdeduc2 hff19r indhhinc indhhin2 /// education, income 
			  had1 diq010 hae2 bpq020 har1 smq020 kiq020 kiq022 /// q&a 
			  pepmnk1r bpxsar bpxsy1 pepmnk5r bpxdar bpxdi1 bmpbmi bmxbmi /// exam 
			  ghp lbxgh sgpsi lbxsgl ubp urp urxuma urxucr cep lbdscrsi // lab
     lab dat "NHANES 1988-2018, Demographic & Health Characteristics"
     save "08_nhmerge$date", replace
	 
}
 	
	
quietly {
	
		 //big data!
	 noi di ""
	 noi di "rows x columns:"
	 noi count
	 ds 
	 noi di wordcount(r(varlist))
	 
     global finish  = clock(c(current_time),"hms")/10^3
     global duration = round(($finish - $start)/60,1)

     noi di "runtime = $duration minute(s)"	

     log close
	
}



```

---

4. 10_nhmort_final.do

```stata

cls
clear

*! mortality linkage
*! cruxes at lines 35 & 47



quietly {
	
     capture log close
     global date: di %td date(c(current_date),"DMY")
     log using "12_nhmort$date.log", replace
     version 12

     global start = clock(c(current_time),"hms")/10^3	

     //1.data
     global nchs https://ftp.cdc.gov/pub/Health_Statistics/NCHS/
     global linkage datalinkage/linked_mortality/
	 global nh3 NHANES_III
     global mort _MORT_2019_PUBLIC.dat
     global varlist ///
         seqn 1-6 ///
         eligstat 15 ///
	     mortstat 16 ///
	     ucod_leading 17-19 ///
         diabetes 20 ///
	     hyperten 21 ///
	     permth_int 43-45 ///
	     permth_exm 46-48 
	 
	 //1988-1994
	 infix $varlist using $nchs$linkage$nh3$mort, clear
	 replace seqn=-1*seqn
	 tempfile dat0
	 save `dat0'
     
	 //1999-2018
     local N=1

     foreach y of numlist 1999 2001 2003 2005 2007 2009 2011 2013 2015 2017 {
         local yplus1=`y' + 1  
         	
	     local nhanes NHANES_`y'_`yplus1'
         infix $varlist using $nchs$linkage`nhanes'$mort, clear
	     tempfile dat`N'
	     save `dat`N''
	     local N=`N'+1
     }
	 
	 use `dat0', clear
     forvalues i=1/10 {
	
	     append using `dat`i''
		 
     }
	
	
}


quietly {
	
	 //2.values
     lab def premiss .z "Missing"
     lab def eligfmt 1 "Eligible" 2 "Under age 18, not available for public release" 3 "Ineligible" 
     lab def mortfmt 0 "Assumed alive" 1 "Assumed deceased" .z "Ineligible or under age 18"
     lab def flagfmt 0 "No - Condition not listed as a multiple cause of death" ///
	                      1 "Yes - Condition listed as a multiple cause of death"	///
						  .z "Assumed alive, under age 18, ineligible for mortality follow-up, or MCOD not available"
     lab def qtrfmt 1 "January-March" ///
	                     2 "April-June" ///
						 3 "July-September" ///
						 4 "October-December" ///
						 .z "Ineligible, under age 18, or assumed alive"
     lab def dodyfmt .z "Ineligible, under age 18, or assumed alive"
     lab def ucodfmt 1 "Diseases of heart (I00-I09, I11, I13, I20-I51)"                           
     lab def ucodfmt 2 "Malignant neoplasms (C00-C97)"                                            , add
     lab def ucodfmt 3 "Chronic lower respiratory diseases (J40-J47)"                             , add
     lab def ucodfmt 4 "Accidents (unintentional injuries) (V01-X59, Y85-Y86)"                    , add
     lab def ucodfmt 5 "Cerebrovascular diseases (I60-I69)"                                       , add
     lab def ucodfmt 6 "Alzheimer's disease (G30)"                                                , add
     lab def ucodfmt 7 "Diabetes mellitus (E10-E14)"                                              , add
     lab def ucodfmt 8 "Influenza and pneumonia (J09-J18)"                                        , add
     lab def ucodfmt 9 "Nephritis, nephrotic syndrome and nephrosis (N00-N07, N17-N19, N25-N27)"  , add
     lab def ucodfmt 10 "All other causes (residual)"                                             , add
     lab def ucodfmt .z "Ineligible, under age 18, assumed alive, or no cause of death data"      , add
	 
	 
	 replace mortstat = .z if mortstat >=.
     replace ucod_leading = .z if ucod_leading >=.
     replace diabetes = .z if diabetes >=.
     replace hyperten = .z if hyperten >=.
     replace permth_int = .z if permth_int >=.
     replace permth_exm = .z if permth_exm >=.
	
}

quietly {
	
	 //3.define
	 
     label var seqn "NHANES Respondent Sequence Number"
     label var eligstat "Eligibility Status for Mortality Follow-up"
     label var mortstat "Final Mortality Status"
     label var ucod_leading "Underlying Cause of Death: Recode"
     label var diabetes "Diabetes flag from Multiple Cause of Death"
     label var hyperten "Hypertension flag from Multiple Cause of Death"
     label var permth_int "Person-Months of Follow-up from NHANES Interview date"
     label var permth_exm "Person-Months of Follow-up from NHANES Mobile Examination Center (MEC) Date"

     label values eligstat eligfmt
     label values mortstat mortfmt
     label values ucod_leading ucodfmt
     label values diabetes flagfmt
     label values hyperten flagfmt
     label value permth_int premiss
     label value permth_exm premiss

     noi des 
	 
	 //eligibility
	 drop if inlist(eligstat,2,3)
	 duplicates drop 

	 keep seqn mortstat permth_int permth_exm 
	 save "11_nhmort$date", replace 
	 
	 //big data!
	 noi di ""
	 noi di "rows x columns:"
	 noi count
	 ds 
	 noi di wordcount(r(varlist))
	 
     global finish  = clock(c(current_time),"hms")/10^3
     global duration = round(($finish - $start)/60,1)

     noi di "runtime = $duration minute(s)"	 
	 
	 log close
}


```

---

5. 13_nhsurv_final.do

```stata
cls
clear

*! baseline 08_nhmerge.dta (previously 08_nhtable1.dta)
*! outcome 15_nhcohort.dta (previously 15_nhsurv.dta)
*! should be able to produce a km curve 30y mortality

quietly {
	
     capture log close
     global date: di %td date(c(current_date),"DMY")
     log using "14_nhcohort$date.log", replace
     version 15

     global start = clock(c(current_time),"hms")/10^3
	 
	 //nhanes at baseline
	 use 08_nhmerge$date, clear
	 
	 //nhanes at death or censoring
	 noi merge 1:1 seqn using 11_nhmort$date, keep(matched) nogen
	 
	 //variable labels derived from wwwn.cdc.gov/nchs/nhanes/
	 lab var seqn "Sequence number (seqn x -1 for NHANES III)"
	 lab var dmpstat "Examinmation/interview status"
	 lab var dmarethn "Race/ethnicity"
	 lab var hssex "Sex"
	 lab var hsageir "Age at interview (screener) -qty"
	 lab var hfa8r "Education level"
	 lab var hff19r "Total family 12 month income group"
	 lab var pepmnk1r "Overall average K1, systolic, BP(age 5+)"
	 lab var pepmnk5r "Overall average K5, diastolic, BP(age5+)"
	 lab var bmpbmi "Body mass index"
	 lab var ghp "Glycated hemoglobin: %"
	 lab var ubp "Urinary albumin (ug/mL)"
	 lab var urp "Urinary creatinine (mg/dL)"
	 lab var sgpsi "serum glucose: SI (mmol/L)"
	 lab var cep "Serum creatine (mg/dL)"
	 lab var had1 "Ever been told you have sugar/diabetes"
	 lab var hae2 "Doctor ever told had hypertension/HBP"
	 lab var har1 "Have tiy snijed 100_ cigarettes in life"
	 
     lab dat "NHANES 1988-2018, Original Variables"
	 save "15_nhcohort$date", replace 
	 
	 //big data!
	 noi di ""
	 noi di "rows x columns:"
	 noi count
	 ds 
	 noi di wordcount(r(varlist))
	 
     global finish  = clock(c(current_time),"hms")/10^3
     global duration = round(($finish - $start)/60,1)

     noi di "runtime = $duration minute(s)"		 

	 log close

}

des 

```

6. 17_nhtable1_final.do


```stata

cls
macro drop _all

*! inputs 15_nhcohort$date.dta (previously 15_nhsurv$date.dta) 
*! renames cdc/nchs variables (previously done in 07_nhtable1.do)
*! applies selection criteria (previously done in 07_nhtable1.do)
*! ver 1.0 of 17_*.do 02/01/23 (after submission to rdc/nchs/cdc)

quietly {
	
	 capture log close
     global date: di %td date(c(current_date),"DMY")
     log using "19_nhtable1$date.log", replace
     version 12
     global start = clock(c(current_time),"hms")/10^3 
	
     //1.data as sent to rdc/nchs/cdc
     use 15_nhcohort$date, clear  


}

quietly { //var renames
	 
	 //2.variables as used in analysis
	 
	 //seqn
	 isid seqn
	 
	 //survey
	 g year=surv
	 lab var year "Year of Donation, %"
	 label define Year ///
	     1 "1988-1998" ///
		 2 "1999-2000" ///
		 3 "2001-2002" ///
		 4 "2003-2004" ///
		 5 "2005-2006" ///
		 6 "2007-2008" ///
		 7 "2009-2010" ///
		 8 "2011-2012" ///
		 9 "2013-2014" ///
		 10 "2015-2016" ///
		 11 "2017-2018" 
	 label values year Year
	 
	 //eligibility
	     //1988-1998
	 drop if inlist(dmpstat,1,3)
	 
	     //1999-2018
	 drop if ridstatr==1
	 
	 //race
	     //1988-1998
	 g race=dmarethn
	 label var race "Race/ethnicity, %"
	 label define Race 1 "White" 2 "Black" 3 "Hispanic" 4 "Other" 
	 label values race Race
	 
	    //1999-2018
	 recode ridreth1 (3=1)(4=2)(1 2=3)(5=4)
	 replace race=ridreth1 if inrange(year,2,11)
	 
	 recode ridreth3 (3=1)(4=2)(1 2=3)(6=4)(7=5),gen(race2)
	 label var race2 "Race/ethnicity, %"
	 label define Race2 1 "White" 2 "Black" 3 "Hispanic" 4 "Asian" 5 "Other" 
	 label values race2 Race2
	 
	 //sex
	     //1988-1998
	 recode hssex (1=0)(2=1),gen(female)
	 label var female "Female, %"
	 
	     //1999-2018
	 recode riagendr (1=0)(2=1)
	 replace female=riagendr if inrange(year,2,11)
	 
	 //age
	     //1988-1998
     g age=hsageir 
	 label var age "Age, y, median [IQR]"
	 
	     //1999-2018
	 replace age=ridageyr if inrange(year,2,11)
	 
	     //eligibility
	 drop if !inrange(age,18,90)
	 
	 //education
	     //1988-1998
     recode hfa8r ///  
     (0/8=1) ///k-8
	 (9/12 13=2) ///highsch, diploma
	 (14/15=3) ///some coll/assoc
	 (16/17=4) ///coll/above
	 (66/99=.), gen(educ)
	 
	     //1999-2018
	 recode dmdeduc2 ///
	 (2 3=2) ///
	 (4=3) ///
	 (5=4) ///
	 (7 9=.)
	 replace educ=dmdeduc2 if inrange(year,2,11)
	 
     label define Educ /// 
     1 "K-8" ///
	 2 "High school, Diploma/equivalent" ///
	 3 "Some college/associate" ///
	 4 "College graduate/above" 
     label values educ Educ
	 label var educ "Education, %"
		 
	 //income
	     //1988-1998
	 g income=hff19r
	 label var income "Income, $, median [IQR]"
	 
	 replace income=. if inlist(income,88,99) 
     replace income=round(rweibull(2, 5000, 0)) if inrange(income,0,2)
     
	 forvalues i=3/20 {
	     replace income=round(runiform((`i'000-1000),`i'000)) if income==`i'
     }
    
	 local lb=20000
     forvalues i=21/26 {
	     local ub=`lb' + 4999
	     di "`lb'" " `ub'"
	     replace income=round(runiform(`lb',`ub')) if income==`i'
	     local lb=`ub' + 1
     }
   
     replace income=round(rweibull(1.09, 25000, 50000)) if income==27
	 
	     //1999-2018
		 
	 replace income=indhhinc  if inrange(year,2,5)
	 replace income=indhhin2  if inrange(year,6,11)
	 local lb=0
     foreach i of numlist 1 2 3 4 5 {
         local ub=`lb' + 4999	     
		 replace income=runiform(`lb',`ub') if income==`i' & year!=1
		 local lb=`ub' + 1
     }
    
	 local lb=25000
     foreach i of numlist 6 7 8 9 10 11 {
	     local ub=`lb' + 9999
	     replace income=runiform(`lb',`ub') if income==`i' & year!=1
	     local lb=`ub' + 1
     }
   
	 //dm
	     //1988-1998
	 recode had1 (2=0)(8 9=.), gen(dm)  
	 label var dm "Diabetes, %"
	 
	     //1999-2018
	 recode diq010 (1 3=1)(2=0)(7 9=.)
	 replace dm=diq010 if inrange(year,2,11)
		 
	 //htn
	     //1988-1998
	 recode hae2 (2=0)(8 9=.), gen(htn)  
	 label var htn "Hypertension, %"
	 
	     //1999-2018
     recode bpq020 (2=0)(9=.)
	 replace htn=bpq020 if inrange(year,2,11)
	 
	 //ckd 
	     //1988-1998
	
		   
		 //1999-2018
	 recode kiq020 (2=0)(7 9=.) if inlist(year,2),gen(ckd)
	 recode kiq022 (2=0)(7 9=.) if inrange(year,3,11)
	 replace ckd=kiq022 if inrange(year,3,11)
	 label var ckd "CKD, %"
	 
	 //smoke
	     //1988-1998
	 recode har1 (2=0),gen(smk)
	 label var smk "Smoke, %"
	 
	     //1999-2018
	 recode smq020 (2=0)(7 9=.)
	 replace smk=smq020 if inrange(year,2,11)
	 
	 //bp
	     //1988-1998
	 g sbp=pepmnk1r
	 label var sbp "SBP, mmHg, median [IQR]"
	 g dbp=pepmnk5r
	 label var dbp "DBP, mmHg, median [IQR]"
	 
	     //1999-2018
	 replace sbp=bpxsar if inrange(year,2,4)
	 replace dbp=bpxdar if inrange(year,2,4)
	 
	 replace sbp=bpxsy1 if inrange(year,5,11)
	 replace dbp=bpxdi1 if inrange(year,5,11)

	 //bmi
	     //1988-1998
	 g bmi=bmpbmi
	 label var bmi "BMI, kg/m2, median [IQR]"
	 
	     //1999-2018
	 replace bmi=bmxbmi if inrange(year,2,11)

	 //hba1c
	     //1988-1998
	 g hba1c=ghp
	 label var hba1c "HBA1c, %, median [IQR]"
	 
	     //1988-1998
	 replace hba1c=lbxgh if inrange(year,2,11)
	 
	 //uacr
	     //1988-1998
	 g acr=(ubp*1/10^-3)/(urp*1/10^-1) 
	 label var acr "uACR, mg/g, median [IQR]"
	 
	     //1999-2018
     replace acr=(urxuma*1/10^-3)/(urxucr*1/10^-1) if inrange(year,2,11)
	
	g logacr=log(acr)
	 label var logacr "Log uACR, log(mg/g), median [IQR]"	
	 
	 //glucose
	     //1988-1998
	 g glucose=sgpsi*18/10
	 label var glucose "Glucose, mg/L, median [IQR]"
	 
	     //1999-2018
	 replace glucose=lbxsgl*1/10^1 if inrange(year,2,11)
	 
	 //creatinine
	     //1988-1998
	 g creat=cep
	 label var creat "Serum Creatinine, mg/dL, median [IAR]"
	 
	     //1999-2018
	 replace creat=lbdscrsi/88.42 if inrange(year,2,11)

     
	 //ckd-epi
     replace creat=0.960*creat-0.184 if year==1
     local k=cond(female==1,0.7,0.9)
     local alpha=cond(female==1,-0.329,-0.411)
     g egfr=141*min(creat/`k',1)^`alpha'* ///
	            max(creat/`k',1)^-1.209* ///
			    0.993^age* ///
				1.018^female* ///
				1.159^(race==2) if !missing(creat)
 	 label var egfr "eGFR, ml/min/1.73m2, median [IAR]"
	 
	
}

quietly {
	
	 //3.keep variables 
	     keep seqn age female race race2 educ income dm htn smk ///
		      sbp dbp bmi hba1c glucose acr logacr creat egfr year ///
			  mortstat permth_int permth_exm
}


     lab dat "NHANES 1988-2018, Renamed Variables"
     save "18_nhtable1$date", replace
	
	
quietly {
	
		 //big data!
	 noi di ""
	 noi di "rows x columns:"
	 noi count
	 ds 
	 noi di wordcount(r(varlist))
	 
     global finish  = clock(c(current_time),"hms")/10^3
     global duration = round(($finish - $start)/60,1)

     noi di "runtime = $duration minute(s)"	

     log close
	
}



```

7. 20_nhsas7bdat.do

```stata
* https://chat.openai.com/chat
* how do I create a .sas7bdat format file from stata?
* and the rest is history :)

use 18_nhtable102feb2023, clear
outsheet using "21_nhtable102feb2023.csv", comma replace

if 0 {
	
	 proc import datafile='21_nhtable102feb2023.csv'
     out=your_sas_dataset
     dbms=csv replace;
     getnames=yes;
     run;

	
}

```